# Embedding word2vec fasttext

## Задача поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)
3. Проверить насколько хорошо работают подходы

## Общая подготовка (импорт библиотек, модулей, ...)

In [2]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32920 sha256=f23a06106fd5ea560f4666b801ca3867e227d48d0add53a5ede8cb3d29d2fde7
  Stored in directory: c:\users\данила\appdata\local\pip\cache\wheels\eb\03\0d\3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop-words


In [4]:
!pip install annoy

  Created wheel for annoy: filename=annoy-1.17.0-cp38-cp38-win_amd64.whl size=53013 sha256=35a649d41387c3e61bbea725ec10f6ca97c26809aade023dfb07476fe1bb5695
  Stored in directory: c:\users\данила\appdata\local\pip\cache\wheels\77\01\de\4421524f9997a25dfa7291121565d12ef514154945e80e907a
Successfully built annoy


In [20]:
import pandas as pd
import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
import annoy
import numpy as np
import gensim.models
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

## Загрузка и подготовка данных

In [3]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)
df.tail()

,text,label
111918,Но не каждый хочет что то исправлять:( http://...,negative
111919,скучаю так :-( только @taaannyaaa вправляет мо...,negative
111920,"Вот и в школу, в говно это идти уже надо(",negative
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",negative
111922,Такси везет меня на работу. Раздумываю приплат...,negative


In [6]:
#разбиваем данные на train и test
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [4]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [5]:
# Функция преобразования твита (лемматизация, удаление стоп-слов и пунктуации)
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [7]:
# Процедура подготовки данных к обучению
assert True

# Preprocess for models fitting
tweet_tokens = []
c = 0
for text in x_train:
    tweet_prep = preprocess_txt(text)
    tweet_tokens.append(tweet_prep)
    c += 1
    if c > 100000:
        break

In [8]:
# Функция поиска 5 ближайших твитов
def get_response(tweet, index, model, index_map):
    tweet_token = preprocess_txt(tweet)
    vector = np.zeros(300)
    norm = 0
    for word in tweet_token:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    similar_tweet = index.get_nns_by_vector(vector, 5, include_distances=True)
    
    result = pd.DataFrame({'tweet': [index_map[i]  for i in similar_tweet[0]], 
                           'distance' : similar_tweet[1]})
    return result

In [9]:
# Процедура поиска ближайших твитов
def get_model_index(model):
    model_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

    index_map = {}
    counter = 0

    for tweet in x_train:
        n_model = 0
        index_map[counter] = tweet
        tweet_prep = preprocess_txt(tweet)
        
        vector_model = np.zeros(300) #вектор твита

        for word in tweet_prep:
            if word in model:
                vector_model += model[word]
                n_model += 1
        if n_model > 0:
            vector_model = vector_model / n_model #нормализуем вектор твита
       
        model_index.add_item(counter, vector_model) #добавляем элемент для индексации
     
        counter += 1

        if counter > 100000:
            break

    model_index.build(10) #из индексов создаем лес из 10 деревьев
    return model_index, index_map

## Word2wec

In [10]:
modelW2V = Word2Vec(sentences=tweet_tokens, vector_size=300, window=5, min_count=1)

In [11]:
w2v_index, index_map = get_model_index(modelW2V.wv)

In [12]:
print(x_test.values[0])
get_response(x_test.values[0], w2v_index, modelW2V.wv, index_map)

@dailykiselev @pavelsheremet это сердечко из воздушного шарика )))


,tweet,distance
0,@meh_joey кстати фотика не будет :с\n:(:(:(:(...,0.024587
1,"Сегодня мне снилось что я профессор,хожу по ка...",0.025040
2,@99Umka99 @UporotyhpieDean @v_vladman_v @Minut...,0.025135
3,RT @rockgirlirushka: Первый день зимы....начин...,0.026439
4,RT @barbi_risha: большой компанией кататься па...,0.026507


## Fasttext

In [13]:
modelFT = FastText(sentences=tweet_tokens, vector_size=300, min_count=1, window=5, workers=8)

In [14]:
ft_index, index_map = get_model_index(modelFT.wv)

In [15]:
print(x_test.values[0])
get_response(x_test.values[0], ft_index, modelFT.wv, index_map)

@dailykiselev @pavelsheremet это сердечко из воздушного шарика )))


,tweet,distance
0,"Brothers: A tale of two sons классная, только ...",0.015430
1,@lentaruofficial Это же Святая София Константи...,0.017493
2,#HappyBirthdayTreCool\n#HappyBirthdayTreCool\n...,0.017877
3,@circumstances_ @ValeriaNefedowa типичная реак...,0.019090
4,@Vafudr бясплатная медыцына гарантавана для ўс...,0.021530


## Embedding слой

In [21]:
text_dataset = tf.data.Dataset.from_tensor_slices([x for l in tweet_tokens for x in l]) #список всех токенов из x_train
#создание слоя словаря
vectorize_layer = TextVectorization(
    max_tokens=100000,
    output_mode='int',
    output_sequence_length=1)
#создаем словарь
vectorize_layer.adapt(text_dataset)

#создаем модель
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string)) #на вход подается строка
model.add(vectorize_layer)  #получаем тензор (1, 100), содержащий для каждого слова индексы из словаря 
model.add(Embedding(100000, 300, input_length=100, mask_zero=True))

In [22]:
print(x_test.values[0])
v1 = model.predict([x_test.values[0]])[0][0]
v2 = model.predict([x_test.values[1]])[0][0]
v1, v1.shape

@dailykiselev @pavelsheremet это сердечко из воздушного шарика )))


(array([-4.25471552e-02,  2.71144174e-02, -2.19220519e-02, -3.42767462e-02,
         2.67246030e-02,  1.23402588e-02, -1.06560104e-02,  3.93012911e-03,
         1.92700699e-03,  4.27275412e-02, -4.34834845e-02,  4.18946631e-02,
        -8.29537958e-03,  3.84259261e-02,  3.03425901e-02,  4.26512994e-02,
         9.12897661e-03,  3.29642333e-02,  2.08178051e-02, -6.90040737e-03,
        -5.12353331e-03,  1.61911510e-02, -3.28320265e-02, -2.93199662e-02,
        -3.34843174e-02,  2.55197771e-02, -1.38547309e-02, -5.51538542e-03,
         2.35513337e-02,  3.92812230e-02, -3.14476639e-02,  2.77283899e-02,
         4.73274253e-02,  4.69980575e-02,  2.18016393e-02, -7.86186382e-03,
        -3.70436683e-02,  1.18833557e-02, -4.30220366e-03, -1.52326748e-03,
         3.69067080e-02,  1.98737532e-03, -3.53524573e-02,  3.75496037e-02,
         3.90128605e-02,  2.12968849e-02,  1.94350630e-03,  4.24803831e-02,
        -3.24489474e-02,  2.20609829e-03,  8.69869068e-03,  8.98983330e-03,
        -3.9

In [23]:
# Поиск ближайших твитов
emb_index = annoy.AnnoyIndex(300 ,'angular') # поиск ближайших соседей

emb_map = {}
counter = 0

for tweet in x_train:
    emb_map[counter] = tweet
    vector_emb = model.predict([tweet])[0][0]
    emb_index.add_item(counter, vector_emb) # добавляем элемент для индексации
    counter += 1
        
    if counter > 100000:
        break

emb_index.build(10) # из индексов создаем лес из 10 деревьев

True

In [24]:
def get_response_emb(model, emb_index, tweet, n_similar_tweets):
    vector = model.predict([tweet])[0][0]

    similar_tweet = emb_index.get_nns_by_vector(vector, n_similar_tweets, include_distances=True)
    
    result = pd.DataFrame({'tweet': [emb_map[i]  for i in similar_tweet[0]], 
                           'distance' : similar_tweet[1]
                          })
    return result

In [25]:
print(x_test.values[0])
get_response_emb(model, emb_index, x_test.values[0], 5)

@dailykiselev @pavelsheremet это сердечко из воздушного шарика )))


,tweet,distance
0,"@mint_muffin ахахха, ты это зряяя\nты не предс...",1.290093
1,"@satanishka666 согласна!\nи да, только подумал...",1.290618
2,"@satanishka666 дада)\nну, нада постараться!\nJ...",1.290618
3,@love__AndyBVB ничего((99((9((9((((9 жизнь пря...,1.298755
4,@SimbIblis зашибись.. Стагнировали бы еще нало...,1.323780
